In [2]:
import mir3.modules.features as feat
import mir3.modules.tool.wav2spectrogram as spec
import mir3.modules.features.centroid as cent
import mir3.modules.features.rolloff as roll
import mir3.modules.features.flatness as flat
import mir3.modules.features.flux as flux
import mir3.modules.features.mfcc as mfcc
import mir3.modules.features.diff as diff
import mir3.modules.features.stats as stats
reload(stats)
import mir3.modules.features.join as join
import mir3.modules.tool.to_texture_window as tex
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import copy
from scipy.stats import friedmanchisquare as friedman
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import ttest_ind as ttest
from scipy.stats import ttest_ind_from_stats as ttest2
from scipy.stats import f_oneway as f_oneway
from matplotlib import pyplot as plt
import numpy as np
import csv


def dataset_characterization(dataset_file):
    dataset = {}
    with open(dataset_file, 'rb') as f:
        reader = csv.reader(f)
        for row in reader:
        	#arrange the dataset (dictionary) with index as filename and content as its emotion
            dataset[row[0]] = row[2]
    return dataset

def dataset_class_histogram(dataset):
    histogram = {}
    for data in dataset:
        if dataset[data] not in histogram:
            histogram[dataset[data]] = 1
        else:
            histogram[dataset[data]] += 1
    return histogram

def genre_characterization(dataset_file):
    genre = {}
    with open(dataset_file, 'rb') as f:
        reader = csv.reader(f)
        for row in reader:
	        #arrange the dataset (dictionary) with index as filename and content as its genre
            genre[row[0]] = row[1]
    return genre



In [3]:

def features_gtzan(filename, directory=""):
    # Calculate spectrogram (normalizes wavfile)
    converter = spec.Wav2Spectrogram()
    print (directory+filename)
    s = converter.convert(open(directory + filename), window_length=2048, dft_length=2048,
                window_step=1024, spectrum_type='magnitude', save_metadata=True)

    # Extract low-level features, derivatives, and run texture windows

    d = diff.Diff()
    features = (cent.Centroid(), roll.Rolloff(), flat.Flatness(), flux.Flux(), mfcc.Mfcc())

    all_feats = None
    for f in features:
        track = f.calc_track(s) # Feature track
        all_feats = join.Join().join([all_feats, track])
        dtrack = d.calc_track(track) # Differentiate
        all_feats = join.Join().join([all_feats, dtrack])
        ddtrack = d.calc_track(dtrack) # Differentiate again
        all_feats = join.Join().join([all_feats, ddtrack])

        # Texture window
        t = tex.ToTextureWindow().to_texture(all_feats, 40)

    #Statistics
    s = stats.Stats()
    d = s.stats([t], mean=True, variance=True)
    return d



def low_level_features(dataset_file, dataset_dir=""): # Estimate low-level features.
                                                      # Returns sklearn-compatible structures.
    d = dataset_characterization(dataset_file)
    g = genre_characterization(dataset_file)
    labels = []
    genres = []
    features = []
    for f in d:
        feat = features_gtzan(filename=f, directory=dataset_dir)
        features.append(feat.data)
        labels.append(d[f])
        genres.append(g[f])
        #print(features, labels, genres)

    #returns: labels are its precieved emotions and genres are genres
    return features, labels, genres


In [4]:

def label_to_numbers(labels, d):
    return [d.keys().index(i) for i in labels]

def numbers_to_labels(numbers, d):
    return [d.keys[i] for i in numbers]


In [5]:

def f1_from_confusion(c):
    ret = []
    for i in xrange(c.shape[0]):
        n_i = np.sum(c[i,:])
        est_i = np.sum(c[:,i])
        if n_i > 0:
            R = c[i,i] / float(n_i)
        else:
            R = 0.0
        if est_i > 0:
            P = c[i,i] / float(est_i)
        else:
            P = 0.0

        if (R+P) > 0:
            F = 2*R*P/(R+P)
        else:
            F = 0.
        ret.append([R, P, F])
    return ret



In [6]:


def model_comparison(features, labels, models, parameters_to_optimize, genres = None):

    features = np.array(features)

    #normalizing features
    #features = normalize(features)
    print("normed feats\n")
    #print(normed_features)

    #use genre as a parameter
    if genres is not None:
        genres = np.array(genres)
        #print("\n dentro do aprendizado, genres:\n")

        genres.shape = (len(genres),1)
        genres_matrix = np.zeros((len(genres), 5))
        for i in xrange(len(genres)):
            genres_matrix[i][genres[i]] = 1;
       
        features = np.hstack((features, genres_matrix))
       

    skf = StratifiedKFold(labels, n_folds=10)

    f1 = np.zeros((10,len(models)))
    r = np.zeros((10,len(models)))
    p = np.zeros((10,len(models)))

    for m in xrange(len(models)):
        n = 0
       	conf_mat_total = None
        for train_index, test_index in skf:
            X_train, X_test = features[train_index,:], features[test_index,:]
            Y_train, Y_test = [labels[i] for i in train_index], [labels[i] for i in test_index]
            scaler = preprocessing.StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

            cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0) # 80% train / 20% validation
            classifier = GridSearchCV(estimator=copy.deepcopy(models[m]), cv=cv, param_grid=parameters_to_optimize[m], scoring='f1_weighted')

            classifier.fit(X_train, Y_train)
            Y_pred = classifier.predict(X_test)
            confusion = confusion_matrix(Y_test, Y_pred)
            if conf_mat_total is None:
            	conf_mat_total = confusion
            else:
            	conf_mat_total += confusion
            conf = f1_from_confusion(confusion)
            conf_all = np.average(conf, axis=0)

            r[n,m] = conf_all[0]
            p[n,m] = conf_all[1]
            f1[n,m] = conf_all[2]
            n += 1
        print("\nconf_mat:\n")
        print(conf_mat_total)

    return r, p, f1


In [7]:

def model_comparison_sing_gen_mat(features, labels, models, parameters_to_optimize, genres = None):

    features = np.array(features)

    #normalizing features
    #features = normalize(features)
    print("normed feats\n")
    #print(normed_features)

    #use genre as a parameter
    if genres is not None:
        genres = np.array(genres)
        #print("\n dentro do aprendizado, genres:\n")

        genres.shape = (len(genres),1)
        genres_matrix = np.zeros((len(genres), 5))
        for i in xrange(len(genres)):
            genres_matrix[i][genres[i]] = 1;
       
        features = np.hstack((features, genres_matrix))
       

    skf = StratifiedKFold(labels, n_folds=10)

    f1 = np.zeros((10,len(models)))
    r = np.zeros((10,len(models)))
    p = np.zeros((10,len(models)))

    for m in xrange(len(models)):
        n = 0
       	conf_mat_total = None
        for train_index, test_index in skf:
            X_train, X_test = features[train_index,:], features[test_index,:]
            Y_train, Y_test = [labels[i] for i in train_index], [labels[i] for i in test_index]
            scaler = preprocessing.StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

            cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0) # 80% train / 20% validation
            classifier = GridSearchCV(estimator=copy.deepcopy(models[m]), cv=cv, param_grid=parameters_to_optimize[m], scoring='f1_weighted')

            classifier.fit(X_train, Y_train)
            Y_pred = classifier.predict(X_test)
            confusion = confusion_matrix(Y_test, Y_pred)
            if conf_mat_total is None:
            	conf_mat_total = confusion
            else:
            	conf_mat_total += confusion
            conf = f1_from_confusion(confusion)
            conf_all = np.average(conf, axis=0)

            r[n,m] = conf_all[0]
            p[n,m] = conf_all[1]
            f1[n,m] = conf_all[2]
            n += 1
        print("\nconf_mat:\n")
        print(conf_mat_total)

    return r, p, f1

In [8]:

def dataset_characterization_single_genre(dataset_file, genre):
    dataset = {}
    print(genre)
    with open(dataset_file, 'rb') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[1] == genre:
                #arrange the dataset (dictionary) with index as filename and content as its emotion
                dataset[row[0]] = row[2]
    return dataset

def dataset_class_histograms(dataset):
    histogram = {}
    for data in dataset:
        if dataset[data] not in histogram:
            histogram[dataset[data]] = 1
        else:
            histogram[dataset[data]] += 1
    return histogram

def genre_characterization_single_genre(dataset_file, genre):
    genre = {}
    with open(dataset_file, 'rb') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[1] == genre:
                #arrange the dataset (dictionary) with index as filename and content as its genre
                genre[row[0]] = row[1]
    return genre

In [9]:
dataset_dir = "./datasets/gtzan/"
dataset_file = 'base.csv'

dataset = dataset_characterization(dataset_file)
#print "dataset"
#print dataset
#print (dataset_class_histogram(dataset))
genre = genre_characterization(dataset_file)
#print"genre"
#print(genre)
print (dataset_class_histogram(genre))

{'Indie': 37, 'Jazz': 36, 'Metal': 37, 'bossa': 37, 'Classical': 35}


In [10]:
features, labels, genre_labels = low_level_features(dataset_file, dataset_dir)

print("ACABOU\n")

gammas = np.logspace(-6, -2, 2)
Cs = np.array([1, 1000, 10000])
n_neighbors_s = np.array([1, 5, 10, 15, 20])

params_knn = dict(n_neighbors=n_neighbors_s)
params_svm = dict(gamma=gammas, C=Cs)
parameters_to_optimize = [params_knn, params_svm]

models = [KNeighborsClassifier(), SVC(class_weight='balanced')]


./datasets/gtzan/Toquinho_-_Meditacao.wav
./datasets/gtzan/Etta_James_-_Stormy_Weather.wav
./datasets/gtzan/aguas_de_Marco.wav
./datasets/gtzan/Is_This_It.wav
./datasets/gtzan/Freedom_Call_-_Warriors.wav
./datasets/gtzan/Bon_Iver_-_Skinny_Love.wav
./datasets/gtzan/Nothing_Else_Matters.wav
./datasets/gtzan/sonata-arctica-letter-to-dana.wav
./datasets/gtzan/Elis_Regina_-_O_Bebado_e_a_Equilibrista.wav
./datasets/gtzan/So_Far_Away.wav
./datasets/gtzan/Villa-Lobos_-_Bachianas_Brasileiras_No_2_-_IV_Tocata_(O_trenzinho_do_caipira).wav
./datasets/gtzan/Chico_Buarque_de_Hollanda_-_Geni_e_o_Zepelim_(1979).wav
./datasets/gtzan/Edward_Sharpe_&_The_Magnetic_Zeros_-_Home.wav
./datasets/gtzan/Bloom.wav
./datasets/gtzan/Only_Ones_Who_Know.wav
./datasets/gtzan/Minha_Namorada_-_Miucha_e_Vinicius_de_Moraes.wav
./datasets/gtzan/p_i_tchaikovsky_trepak_russian_dance_nutcracker_ballet_suite_op_71.wav
./datasets/gtzan/Tony_Bennett_-_Dont_Get_Around_Much_Anymore.wav
./datasets/gtzan/Fade_to_Black.wav
./dataset

/usr/lib/python2.7/dist-packages/scipy/signal/signaltools.py:1741: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = zeros(newshape, 'D')


In [11]:

# #using model_comparison without genre as a feature
print("dataset: ")
print(dataset)
print("labels: ")
print (labels)
print("\n labels to numbers:")
print (label_to_numbers(labels, dataset_class_histogram(dataset)))
print("genre: ")
print (genre_labels)
print("\n labels to numbers:")
print (label_to_numbers(genre_labels, dataset_class_histogram(genre)))

#using model_comparison with genre as a feature
recall, precision, f1_score = model_comparison(features, label_to_numbers(labels, dataset_class_histogram(dataset)), models, parameters_to_optimize)

avg_f1_tudo = np.average(f1_score, axis=0)
std_f1_tudo = np.std(f1_score, axis = 0)
n_tudo = len(f1_score)
#printing results with genre as a parameter
print ("genre is not a feature \n")
print "f1-score medio",  np.average(f1_score, axis=0)
print "desvio padrao", np.std(f1_score, axis = 0)

if len(models) > 2:
#    print [f1_score[:,i].T for i in range(len(models))]
    print "Anova: ", f_oneway( *f1_score.T  )[1]


else:
    #compare results with knn or svc
    print "T-test: knn vs svc", ttest( f1_score[:,0].T,  f1_score[:,1].T)[1]


dataset: 
{'Toquinho_-_Meditacao.wav': 'Sad', 'Etta_James_-_Stormy_Weather.wav': 'Sad', 'aguas_de_Marco.wav': 'Happy', 'Is_This_It.wav': 'Sad', 'Freedom_Call_-_Warriors.wav': 'Happy', 'Bon_Iver_-_Skinny_Love.wav': 'Sad', 'Nothing_Else_Matters.wav': 'Sad', 'sonata-arctica-letter-to-dana.wav': 'Sad', 'Elis_Regina_-_O_Bebado_e_a_Equilibrista.wav': 'Sad', 'So_Far_Away.wav': 'Sad', 'Villa-Lobos_-_Bachianas_Brasileiras_No_2_-_IV_Tocata_(O_trenzinho_do_caipira).wav': 'Happy', 'Chico_Buarque_de_Hollanda_-_Geni_e_o_Zepelim_(1979).wav': 'Sad', 'Edward_Sharpe_&_The_Magnetic_Zeros_-_Home.wav': 'Happy', 'Bloom.wav': 'Sad', 'Only_Ones_Who_Know.wav': 'Sad', 'Minha_Namorada_-_Miucha_e_Vinicius_de_Moraes.wav': 'Happy', 'p_i_tchaikovsky_trepak_russian_dance_nutcracker_ballet_suite_op_71.wav': 'Happy', 'Tony_Bennett_-_Dont_Get_Around_Much_Anymore.wav': 'Happy', 'Fade_to_Black.wav': 'Sad', 'Jorge_Ben_Jor_-_Que_Maravilha.wav': 'Happy', 'Bring_Me_To_Life.wav': 'Sad', 'mozart_-_eine_kleine_nachtmusik_rondo_a


[[69 23]
 [20 70]]

conf_mat:

[[72 20]
 [22 68]]
genre is not a feature 

f1-score medio [ 0.76092062  0.76765787]
desvio padrao [ 0.09840776  0.1084198 ]
T-test: knn vs svc 0.891742017663


/usr/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:

#using model_comparison with genre as a feature
recall, precision, f1_score = model_comparison(features, label_to_numbers(labels, dataset_class_histogram(dataset)), models, parameters_to_optimize, label_to_numbers(genre_labels, dataset_class_histogram(genre)))

#printing results with genre as a parameter
print ("genre is a feature \n")
print "f1-score medio", np.average(f1_score, axis=0)
print "desvio padrao", np.std(f1_score, axis = 0)

if len(models) > 2:
#    print [f1_score[:,i].T for i in range(len(models))]
    print "Anova: ", f_oneway( *f1_score.T  )[1]


else:
    #compare results with knn or svc
    print "T-test: knn vs svc", ttest( f1_score[:,0].T,  f1_score[:,1].T)[1]

normed feats


conf_mat:

[[68 24]
 [20 70]]

conf_mat:

[[73 19]
 [20 70]]
genre is a feature 

f1-score medio [ 0.75393647  0.78412502]
desvio padrao [ 0.11588068  0.08514409]
T-test: knn vs svc 0.536731105602


In [13]:

dataset_dir = "./datasets/gtzan/"
dataset_file = 'base.csv'

# dataset_singe__genre = dataset_characterization_single_genre(dataset_file)
# #print (dataset_class_histogram(dataset))
# genre_single_genre = genre_characterization_single_genre(dataset_file)
# #print (dataset_class_histogram(genre))

#features, labels, genre_labels = low_level_features(dataset_file, dataset_dir)

# print("ACABOU\n")

# gammas = np.logspace(-6, -2, 2)
# Cs = np.array([1, 1000, 10000])
# n_neighbors_s = np.array([1, 5, 10, 15, 20])

# params_knn = dict(n_neighbors=n_neighbors_s)
# params_svm = dict(gamma=gammas, C=Cs)
# parameters_to_optimize = [params_knn, params_svm]

# models = [KNeighborsClassifier(), SVC(class_weight='balanced')]


In [14]:
#create a different dataset for each genre

tested_genres = []

#print genre_labels, len(genre_labels)
#print labels, len(labels)
for single_genre in genre_labels:
    if single_genre not in tested_genres:
        print "single genre", single_genre
        tested_genres.append(single_genre)
        numbers_by_genre = 0
        features_single_genre = []
        labels_single_genre = []
        genre_labels_single_genre = []
        counter = 0
        #copying features and labels for each genre avoiding redundant signal processing 
        for i in genre_labels:
            if i == single_genre:
                numbers_by_genre += 1
                features_single_genre.append(features[counter])
                labels_single_genre.append(labels[counter])
                genre_labels_single_genre.append(genre_labels[counter])
            counter += 1
            
        
        print "single genre", single_genre
        dataset_single_genre = dataset_characterization_single_genre(dataset_file, single_genre)
        #print "dataset_single_genre", dataset_single_genre
        dataset_class_histogram(dataset_single_genre)
        genre_single_genre = genre_characterization_single_genre(dataset_file, single_genre)
        #print (dataset_class_histogram(dataset_single_genre))
        #print "labels single genre", labels_single_genre, len(labels_single_genre)
        #print(label_to_numbers(labels_single_genre, dataset_class_histogram(dataset_single_genre)))
        
        recall_single_genre, precision_single_genre, f1_score_single_genre = model_comparison(features_single_genre, label_to_numbers(labels_single_genre, dataset_class_histogram(dataset_single_genre)), models, parameters_to_optimize)

        #printing results with genre as a parameter
        print ("using genre")
        print (single_genre), numbers_by_genre
        print "\nf1 medio: "
        print np.average(f1_score_single_genre, axis=0)
        print "desvio padrao medio", np.std(f1_score_single_genre, axis=0)
        print "t teste tudo vs genero", ttest2(np.average(f1_score_single_genre, axis=0), np.std(f1_score_single_genre, axis=0), len(f1_score_single_genre), avg_f1_tudo, std_f1_tudo, n_tudo)

        if len(models) > 2:
        #    print [f1_score[:,i].T for i in range(len(models))]
            print "Anova: ", f_oneway( *f1_score_single_genre.T  )[1]


        else:
            #compare results with knn or svc
            print "T-test: knn vs svc", ttest( f1_score_single_genre[:,0].T,  f1_score_single_genre[:,1].T)[1]


single genre bossa
single genre bossa
bossa
normed feats


conf_mat:

[[12  7]
 [ 9  9]]

conf_mat:

[[10  9]
 [10  8]]
using genre
bossa 37

f1 medio: 
[ 0.52666667  0.44166667]
desvio padrao medio [ 0.21694853  0.21848595]
t teste tudo vs genero Ttest_indResult(statistic=array([-3.10957555, -4.22649516]), pvalue=array([ 0.0060512 ,  0.00050746]))
T-test: knn vs svc 0.418409463079
single genre Jazz
single genre Jazz
Jazz
normed feats


conf_mat:

[[ 9  9]
 [ 1 17]]

conf_mat:

[[14  4]
 [ 1 17]]
using genre
Jazz 36

f1 medio: 
[ 0.67        0.85333333]
desvio padrao medio [ 0.26602005  0.21039645]
t teste tudo vs genero Ttest_indResult(statistic=array([-1.01367174,  1.14466661]), pvalue=array([ 0.32417219,  0.26734056]))
T-test: knn vs svc 0.122268201187
single genre Indie
single genre Indie
Indie
normed feats


conf_mat:

[[13  5]
 [ 2 17]]

conf_mat:

[[13  5]
 [ 1 18]]
using genre
Indie 37

f1 medio: 
[ 0.8         0.79333333]
desvio padrao medio [ 0.28751812  0.24303635]
t teste t

/usr/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
